In [1]:
# Initialize notebook
import numpy as np
import json
import plotly.graph_objects as go
from bs4 import BeautifulSoup


METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

ENCOUNTER_DIFFICULTIES = {
    'Easy':   {
        'XP': [25,50,75,125,250,300,350,450,550,600,800,1000,1100,1250,1400,1600,2000,2100,2400,2800], 
        'color': '#1F77B4'
    },
    'Medium': {
        'XP': [50,100,150,250,500,600,750,900,1100,1200,1600,2000,2200,2500,2800,3200,3900,4200,4900,5700], 
        'color': '#FF7F0E'
    },
    'Hard':   {
        'XP': [75,150,225,375,750,900,1100,1400,1600,1900,2400,3000,3400,3800,4300,4800,5900,6300,7300,8500], 
        'color': '#2CA02C'
    },
    'Deadly': {
        'XP': [100,200,400,500,1100,1400,1700,2100,2400,2800,3600,4500,5100,5700,6400,7200,8800,9500,10900,12700], 
        'color': '#D62728'
    },
    'Daily':  {
        'XP': [300,600,1200,1700,3500,4000,5000,6000,7500,9000,10500,11500,13500,15000,18000,20000,25000,27000,30000,40000], 
        'color': '#9467BD'
    },
}

In [2]:
# Setup template and html config for plotly figures.
TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        legend=dict(
            #bordercolor='Black',
            #borderwidth=1,
            font_size=10,
            #tracegroupgap=5,
            #orientation='h',
        )
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}


In [3]:
# Plots average encounter XP for each class on days with only one Deadly encounters and no short rest. 
n_enc = 1
encounter_difficulty = 'Deadly'

with open(f'../xp-and-player-characters/pc data - Medium 2 short rests.json', 'r') as fin:
    pc_data0 = json.load(fin)

with open(f'../xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json', 'r') as fin:
    pc_data1 = json.load(fin)

levels = np.array(range(1,21,1))
enc_XP_budget = 0.5 * np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            autorange=False,
            title_text='encounter XP ratio',
            range=[0.9, 2.1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.04,
            xanchor='left', x=0.01,
            orientation='h',
        )
    )
)

eXPmean = np.zeros(20)
aXPmean = np.zeros(20)
for pc in PC_CLASSES:
    eXP = np.zeros(20)
    for level in range(20):
        encounters = pc_data1[pc][level]['encounters']
        encounters = [x for x in encounters if x['type'] == 'encounter']
        eXP[level] = np.mean([x['XP mean'] for x in encounters[0:n_enc]])

    eXPmean += eXP
    aXP  = np.array([x['encounter XP mean'] for x in pc_data0[pc]])
    aXPmean += aXP

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/aXP,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'lvl %{x}<br>'+
            'enc XP %{y:.2f}'+
            '<extra></extra>'
    ))

eXPmean /= len(PC_CLASSES)
aXPmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eXPmean/aXPmean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-single-encounter-xp-vs-level-{encounter_difficulty.lower()}-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    fig_soup.div['style'] = 'aspect-ratio:600/550'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-single-encounter-xp-vs-level-{encounter_difficulty.lower()}-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'aspect-ratio:600/600'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [4]:
# Plots average encounter XP for each class on days with only one Deadly encounters and no short rest. 
n_enc = 1
encounter_difficulty = 'Deadly'

with open(f'../xp-and-player-characters/pc data - Medium 2 short rests.json', 'r') as fin:
    pc_data0 = json.load(fin)

with open(f'../xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json', 'r') as fin:
    pc_data1 = json.load(fin)

levels = np.array(range(1,21,1))
enc_XP_budget = 0.5 * np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            autorange=False,
            title_text='encounter DPR ratio',
            range=[0.9, 2.1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.04,
            xanchor='left', x=0.01,
            orientation='h',
        )
    )
)

eXPmean = np.zeros(20)
aXPmean = np.zeros(20)
for pc in PC_CLASSES:
    eXP = np.zeros(20)
    for level in range(20):
        encounters = pc_data1[pc][level]['encounters']
        encounters = [x for x in encounters if x['type'] == 'encounter']
        eXP[level] = np.mean([x['effective damage per round mean'] for x in encounters[0:n_enc]])

    eXPmean += eXP
    aXP  = np.array([x['effective damage per round mean'] for x in pc_data0[pc]])
    aXPmean += aXP

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/aXP,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'lvl %{x}<br>'+
            'eff DPR %{y:.2f}'+
            '<extra></extra>'
    ))

eXPmean /= len(PC_CLASSES)
aXPmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eXPmean/aXPmean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-single-encounter-edpr-vs-level-{encounter_difficulty.lower()}-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    fig_soup.div['style'] = 'aspect-ratio:600/550'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-single-encounter-edpr-vs-level-{encounter_difficulty.lower()}-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'aspect-ratio:600/600'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [5]:
# Plots average encounter HP for each class on days with only one Deadly encounters and no short rest. 
n_enc = 1
encounter_difficulty = 'Deadly'

with open(f'../xp-and-player-characters/pc data - Medium 2 short rests.json', 'r') as fin:
    pc_data0 = json.load(fin)

with open(f'../xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json', 'r') as fin:
    pc_data1 = json.load(fin)

levels = np.array(range(1,21,1))
enc_XP_budget = 0.5 * np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            autorange=False,
            title_text='encounter HP ratio',
            range=[0.95, 1.45],
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='bottom',  y=1.04,
            xanchor='left', x=0.01,
            orientation='h',
        )
    )
)

eXPmean = np.zeros(20)
aXPmean = np.zeros(20)
for pc in PC_CLASSES:
    eXP = np.zeros(20)

    for level in range(20):
        encounters = pc_data1[pc][level]['encounters']
        encounters = [x for x in encounters if x['type'] == 'encounter']
        eXP[level] = np.mean([x['effective hit points mean'] for x in encounters[0:n_enc]])

    eXPmean += eXP
    #aXP  = np.array([x['encounter XP mean']/x['effective damage per round mean'] for x in pc_data0[pc]])
    for level in range(20):
        encounters = pc_data0[pc][level]['encounters']
        encounters = [x for x in encounters if x['type'] == 'encounter']
        aXP[level] = np.mean([x['effective hit points mean'] for x in encounters[0:]])
    aXPmean += aXP

    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/aXP,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'lvl %{x}<br>'+
            'eff HP %{y:.2f}'+
            '<extra></extra>'
    ))

eXPmean /= len(PC_CLASSES)
aXPmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eXPmean/aXPmean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-single-encounter-ehp-vs-level-{encounter_difficulty.lower()}-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    fig_soup.div['style'] = 'aspect-ratio:600/550'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-single-encounter-ehp-vs-level-{encounter_difficulty.lower()}-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'aspect-ratio:600/600'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [6]:
# Plots average encounter XP for each class on days with only two Hard encounters and no short rest. 
n_enc = 2
encounter_difficulty = 'Hard'

with open(f'../xp-and-player-characters/pc data - Medium 2 short rests.json', 'r') as fin:
    pc_data0 = json.load(fin)

with open(f'../xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json', 'r') as fin:
    pc_data1 = json.load(fin)


levels = np.array(range(1,21,1))
enc_XP_budget = 0.5 * np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            autorange=False,
            title_text='encounter XP ratio',
            range=[0.9, 1.7],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.04,
            xanchor='left', x=0.01,
            orientation='h',
        )
    )
)

eXPmean = np.zeros(20)
aXPmean = np.zeros(20)
for pc in PC_CLASSES:
    eXP = np.zeros(20)
    for level in range(20):
        encounters = pc_data1[pc][level]['encounters']
        encounters = [x for x in encounters if x['type'] == 'encounter']
        eXP[level] = np.mean([x['XP mean'] for x in encounters[0:n_enc]])

    eXPmean += eXP
    aXP  = np.array([x['encounter XP mean'] for x in pc_data0[pc]])
    aXPmean += aXP
    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/aXP,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'lvl %{x}<br>'+
            'enc XP %{y:.2f}'+
            '<extra></extra>'
    ))

eXPmean /= len(PC_CLASSES)
aXPmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eXPmean/aXPmean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-two-encounters-xp-vs-level-{encounter_difficulty.lower()}-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    fig_soup.div['style'] = 'aspect-ratio:600/550'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-two-encounters-xp-vs-level-{encounter_difficulty.lower()}-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'aspect-ratio:600/600'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [7]:
# Plots average encounter XP for each class on days with only two Deadly encounters and one short rest. 
n_enc = 2
encounter_difficulty = 'Deadly'

with open(f'../xp-and-player-characters/pc data - Medium 2 short rests.json', 'r') as fin:
    pc_data0 = json.load(fin)

with open(f'../xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json', 'r') as fin:
    pc_data1 = json.load(fin)


levels = np.array(range(1,21,1))
enc_XP_budget = 0.5 * np.array(ENCOUNTER_DIFFICULTIES['Daily']['XP'])

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            autorange=False,
            title_text='encounter XP ratio',
            range=[0.9, 1.7],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.04,
            xanchor='left', x=0.01,
            orientation='h',
        )
    )
)

eXPmean = np.zeros(20)
aXPmean = np.zeros(20)
for pc in PC_CLASSES:
    eXP = np.zeros(20)
    for level in range(20):
        encounters = pc_data1[pc][level]['encounters']
        encounters = [x for x in encounters if x['type'] == 'encounter']
        eXP[level] = np.mean([x['XP mean'] for x in encounters[0:n_enc]])

    eXPmean += eXP
    aXP  = np.array([x['encounter XP mean'] for x in pc_data0[pc]])
    aXPmean += aXP
    fig.add_trace(go.Scatter(
        x=levels, 
        y=eXP/aXP,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        marker={'size': 5},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'lvl %{x}<br>'+
            'enc XP %{y:.2f}'+
            '<extra></extra>'
    ))

eXPmean /= len(PC_CLASSES)
aXPmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eXPmean/aXPmean,
    mode='lines', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hoverinfo='skip'
))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-two-encounters-xp-vs-level-{encounter_difficulty.lower()}-large.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    fig_soup.div['style'] = 'aspect-ratio:600/550'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(font=dict(size=10), legend=dict(font_size=8))
    file_name = f'./fig-two-encounters-xp-vs-level-{encounter_difficulty.lower()}-small.html'
    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'aspect-ratio:600/600'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))